In [15]:
import sys

# Print all paths where Python searches for modules
sys.path.append('/Users/isaaclo/Development/pico-scholar/backend/')



In [67]:
import mysql.connector
import os

from sqlalchemy import create_engine, text, URL
from llama_index.core import StorageContext, VectorStoreIndex, Document
from llama_index.vector_stores.tidbvector import TiDBVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.vector_stores.types import MetadataFilters, MetadataFilter
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.vector_stores import FilterCondition
from llama_index.core import Settings

from lamatidb.interfaces.settings_manager import SettingsManager
from lamatidb.interfaces.index_interface import IndexInterface
from lamatidb.interfaces.query_interface import QueryInterface, FilterCondition





**Document Retrieval**

In [68]:
TIDB_HOST='gateway01.us-east-1.prod.aws.tidbcloud.com'
TIDB_USERNAME='6K9ZXKTM8XD1n73.root'
TIDB_PASSWORD='sz0sT1mp4ZiMxmqQ'
TIDB_PORT=4000
TIDB_DB_NAME='scibert_alldata_pico'
VECTOR_TABLE_NAME = "scibert_alldata_fulltext"

In [74]:
index_interface = IndexInterface(TIDB_DB_NAME, VECTOR_TABLE_NAME)
index_interface.load_index_from_vector_store()
index = index_interface.get_index() 

# Create a filter that matches the specific document's metadata
metadata_filters = MetadataFilters(
    filters=[MetadataFilter(key="source", value="16626486", operator="==")], condition=FilterCondition.OR
)
vector_index_retriever = VectorIndexRetriever(
            index=index,
            similarity_top_k=1,
            embedding_model=HuggingFaceEmbedding(model_name="allenai/scibert_scivocab_uncased"),
            filters=metadata_filters if metadata_filters else None,
        )

retrieved_nodes = vector_index_retriever.retrieve(" ")
retrieved_nodes


# Use the filter in the retriever

# Retrieve the document
# retrieved_nodes = retriever.retrieve(" ")
# retrieved_nodes

Time taken to load the index: 0.01 seconds for table scibert_alldata_fulltext


In [ ]:
index_interface = IndexInterface(TIDB_DB_NAME, VECTOR_TABLE_NAME)
index_interface.load_index_from_vector_store()
index = index_interface.get_index() 

# Create a filter that matches the specific document's metadata
metadata_filters = MetadataFilters(
    filters=[MetadataFilter(key="source", value="16626486", operator="==")], condition=FilterCondition.OR
)

retriever = index.as_retriever(
    similarity_top_k=10,  # Retrieve only one document
    filters=metadata_filters
)

chat_engine = index.as_chat_engine(chat_mode="context", verbose=True, retriever_kwargs={"retriever": retriever})


**CHATBOT**\
I think the steps will be as follows: \
Once able to retrieve the document, place the document nodes into a VectorStoreIndex. \
The persist the index and save into local storage? <--- This may not be necessary. \
Rebuild storage context, load the index and set it up as as a chat engine. 

In [75]:
chat_engine.chat("tell me about this document")


OperationalError: (pymysql.err.OperationalError) (2013, 'Lost connection to MySQL server during query')
[SQL: SELECT scibert_alldata_fulltext.id AS scibert_alldata_fulltext_id, scibert_alldata_fulltext.meta AS scibert_alldata_fulltext_meta, scibert_alldata_fulltext.document AS scibert_alldata_fulltext_document, VEC_COSINE_DISTANCE(scibert_alldata_fulltext.embedding, %(VEC_COSINE_DISTANCE_1)s) AS distance 
FROM scibert_alldata_fulltext 
WHERE true = 1 ORDER BY distance ASC 
 LIMIT %(param_1)s]
[parameters: {'VEC_COSINE_DISTANCE_1': '[-0.012477166019380093,0.013587450608611107,-0.0075959740206599236,-0.017128176987171173,0.005940701346844435,0.019551847130060196,-0.013397890143096 ... (32623 characters truncated) ... 07751684635877609,-0.005250158254057169,-0.011285640299320221,-0.008564088493585587,-0.008381297811865807,0.005202768370509148,-0.030058933421969414]', 'param_1': 2}]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [43]:
retrieved_nodes

[]

In [26]:
ids = ["16625992"] # Using this id for testing purposes 
filters = [
    {
        "key": "source", 
        "value": doc_id,
        "operator": "==",
    } for doc_id in ids
]
# Need to create Index object first which store 

chat = index.as_chat_engine()
#sum_query_interface = QueryInterface(index)
#sum_query_interface.configure_retriever(similarity_top_k=1, metadata_filters=filters, condition=FilterCondition.OR) 


Time taken to load the index: 0.01 seconds for table scibert_alldata_fulltext


**-------------------- TESTING SPACE --------------------**


In [52]:
simplified_retriever = index.as_retriever(similarity_top_k=1)
simplified_nodes = simplified_retriever.retrieve("Retrieve a document")

OperationalError: (pymysql.err.OperationalError) (2013, 'Lost connection to MySQL server during query ([Errno 54] Connection reset by peer)')
[SQL: SELECT scibert_alldata_fulltext.id AS scibert_alldata_fulltext_id, scibert_alldata_fulltext.meta AS scibert_alldata_fulltext_meta, scibert_alldata_fulltext.document AS scibert_alldata_fulltext_document, VEC_COSINE_DISTANCE(scibert_alldata_fulltext.embedding, %(VEC_COSINE_DISTANCE_1)s) AS distance 
FROM scibert_alldata_fulltext 
WHERE true = 1 ORDER BY distance ASC 
 LIMIT %(param_1)s]
[parameters: {'VEC_COSINE_DISTANCE_1': '[-0.0067748106084764,-0.030874136835336685,0.005149856209754944,0.0017437012866139412,-0.016137048602104187,0.017549509182572365,-0.00723729794844985 ... (32531 characters truncated) ... 036,-0.01809949427843094,-0.03452403470873833,0.016587037593126297,0.011355932801961899,-0.01838698610663414,-0.01899946853518486,0.0117871705442667]', 'param_1': 1}]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [49]:
# CONNECTION TESTING

# import pymysql
# import os

# connection_params = {
#     "host": os.environ['TIDB_HOST'],
#     "user": os.environ['TIDB_USERNAME'],
#     "password": os.environ['TIDB_PASSWORD'],
#     "database": TIDB_DB_NAME,  # Use the same DB name as in your IndexInterface
#     "port": 4000,
#     "ssl": {
#         "ssl_verify_cert": True,
#         "ssl_verify_identity": True
#     }
# }

# try:
#     connection = pymysql.connect(**connection_params)
#     with connection.cursor() as cursor:
#         # Check if the table exists
#         cursor.execute(f"SHOW TABLES LIKE '{VECTOR_TABLE_NAME}'")
#         if cursor.fetchone():
#             print(f"Table {VECTOR_TABLE_NAME} exists.")
            
#             # Describe the table structure
#             cursor.execute(f"DESCRIBE {VECTOR_TABLE_NAME}")
#             table_structure = cursor.fetchall()
#             print("Table structure:")
#             for column in table_structure:
#                 print(column)
            
#             # Check for 'meta' column
#             cursor.execute(f"SHOW COLUMNS FROM {VECTOR_TABLE_NAME} LIKE 'meta'")
#             meta_column = cursor.fetchone()
#             if meta_column:
#                 print("'meta' column exists with type:", meta_column[1])
#             else:
#                 print("'meta' column does not exist in the table.")
#         else:
#             print(f"Table {VECTOR_TABLE_NAME} does not exist.")
    
#     connection.close()
# except Exception as e:
#     print(f"An error occurred: {e}")


Table scibert_alldata_fulltext exists.
Table structure:
('id', 'varchar(36)', 'NO', 'PRI', None, '')
('embedding', 'vector(768)', 'NO', '', None, '')
('document', 'text', 'YES', '', None, '')
('meta', 'json', 'YES', '', None, '')
('create_time', 'datetime', 'YES', '', 'CURRENT_TIMESTAMP', '')
('update_time', 'datetime', 'YES', '', 'CURRENT_TIMESTAMP', 'DEFAULT_GENERATED on update CURRENT_TIMESTAMP')
'meta' column exists with type: json
